In [1]:
import numpy as np
from PIL import Image
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt
import tifffile as tiff
from skimage import measure
import csv
from scipy.spatial import KDTree
from scipy.spatial import cKDTree
from scipy.ndimage import generic_filter
from pathlib import Path
from tifffile import imwrite

In [2]:
def initProb(directory, extension):
    files = sorted(f for f in os.listdir(directory) if f.endswith(extension))
    stack = tiff.imread([os.path.join(directory, f) for f in files])
    
    if stack.ndim == 4:  # (Z, H, W)
        stack = stack[:,:,:,0]
    return np.moveaxis(stack, [0, 1, 2], [2, 1, 0])

def initStack(directory):
    stack = tiff.imread(directory)
    return np.moveaxis(stack, [0, 1, 2], [2, 1, 0])


In [3]:
def format_vertices(verts):
    formatted_verts = [f"({coord[0]}, {coord[1]}, {coord[2]})" for coord in verts]
    return formatted_verts

In [4]:
def create_dataframe(formatted_verts):
    columns = [
        "Coordinate 1", "Coordinate 2", "Ellipsoid Dimensions", "Tags", 
        "Description", "Segment IDs", "Parent ID", "Type", "ID"
    ]
    rows = []
    for vert in formatted_verts:
        rows.append({
            "Coordinate 1": vert,
            "Type": "Point"
        })
    df = pd.DataFrame(rows, columns=columns)
    
    return df

def export_to_csv(df, filename):
    df.to_csv(filename, index=False)

In [5]:
def export_mesh(prob_stack, filename, step_size=10, level = 0.5):
    verts, faces, _, _ = measure.marching_cubes(prob_stack, level=level, step_size = step_size)
    formatted_verts = format_vertices(verts)
    df = create_dataframe(formatted_verts)
    export_to_csv(df, filename)
    return verts, faces

In [6]:
def filter_probs(probs, threshold=250):
    return (probs > threshold).astype(np.uint8)

In [7]:
def dispImage(img):
    plt.imshow(img, cmap='gray')
    plt.show()

In [8]:
def filter_points_within_distance(set1, set2, distance):
    tree_set2 = cKDTree(set2)
    indices_set1 = tree_set2.query_ball_point(set1, distance)
    filtered_points_set1 = set1[np.array([len(idx) > 0 for idx in indices_set1])]
    
    tree_set1 = cKDTree(set1)
    
    indices_set2 = tree_set1.query_ball_point(set2, distance)
    filtered_points_set2 = set2[np.array([len(idx) > 0 for idx in indices_set2])]
    
    return filtered_points_set1, filtered_points_set2


In [9]:
def transform_points(points, rigid_transform):
    n = points.shape[0]
    points_homogeneous = np.c_[points, np.ones(n)] 
    transformed_points = points_homogeneous @ rigid_transform.T 
    return transformed_points[:, :3]

In [10]:
def export_verts(verts, filename):
    formatted_verts = [f"({coord[0]}, {coord[1]}, {coord[2]})" for coord in verts]
    df = create_dataframe(formatted_verts)
    export_to_csv(df, filename)

In [11]:
def filter_faces_by_points(faces, points, filtered_points):
    filtered_indices = np.where((points[:, None] == filtered_points).all(-1))[0]
    mask = np.isin(faces, filtered_indices).all(axis=1)
    return faces[mask]

In [12]:
def calculate_normal(v1, v2, v3, tolerance=1e-10):
    vec1 = v2 - v1
    vec2 = v3 - v1
    normal = np.cross(vec1, vec2)
    norm = np.linalg.norm(normal)
    
    if norm < tolerance:
        return np.array([0, 0, 0])
    
    normal = normal / norm
    return normal

def angle_between_vectors(v1, v2):
    dot_product = np.dot(v1, v2)
    magnitude_v1 = np.linalg.norm(v1)
    magnitude_v2 = np.linalg.norm(v2)
    angle_rad = np.arccos(dot_product / (magnitude_v1   * magnitude_v2))
    angle_deg = np.degrees(angle_rad)
    return angle_deg

In [13]:
def filter_faces_by_angle(faces, points, angle_threshold=45):
    normals = np.array([calculate_normal(points[face[0]], points[face[1]], points[face[2]]) for face in faces])  
    average_normal = np.mean(normals, axis=0)
    average_normal = average_normal / np.linalg.norm(average_normal)
    
    filtered_faces = []
    for face, normal in zip(faces, normals):
        angle = angle_between_vectors(normal, average_normal)
        if angle <= angle_threshold:
            filtered_faces.append(face)
    
    return np.array(filtered_faces)

In [14]:
def remove_unreferenced_points(faces, points):
    referenced_indices = np.unique(faces)
    filtered_points = points[referenced_indices]
    index_mapping = {old_idx: new_idx for new_idx, old_idx in enumerate(referenced_indices)}
    return filtered_points

In [15]:
def project_points(points, plane_point, plane_normal):
    plane_normal = plane_normal / np.linalg.norm(plane_normal) 
    vector = points - plane_point 
    distance = np.dot(vector, plane_normal) 
    projected_points = points - np.outer(distance, plane_normal) 
    return projected_points

In [16]:
def project_downsampled(points):
    centroid = np.mean(points, axis=0)
    centered_points = points - centroid
    _, _, vh = np.linalg.svd(centered_points, full_matrices=False)
    normal = vh[-1]
    projected_points = project_points(points, centroid, normal)

    return projected_points, centroid, normal

In [17]:
def move_points_towards_center_by_distance(points, normal_vector, affine_matrix, bounding_box_center, distance = 2):
    # Calculate transformation inverse and transform normal vector
    inverse_affine_matrix = np.linalg.inv(affine_matrix)
    untransformed_normal_vector = inverse_affine_matrix[:3, :3] @ normal_vector
    untransformed_normal_vector /= np.linalg.norm(untransformed_normal_vector)  # Normalize

    # Calculate direction vectors to the center
    direction_to_center = bounding_box_center - points
    direction_to_center /= np.linalg.norm(direction_to_center, axis=1, keepdims=True)  # Vectorized normalization

    # Make sure normal vector points towards volume center
    if np.mean(np.dot(direction_to_center, untransformed_normal_vector)) < 0:
        untransformed_normal_vector = -untransformed_normal_vector

    # Move points
    moved_points = points + distance * untransformed_normal_vector

    return moved_points

In [18]:
def plot_projection_as_image(points, projected_points, img_shape, values):
    def rotation_matrix_from_vectors(vec1, vec2):
        vec1 = vec1 / np.linalg.norm(vec1)
        vec2 = vec2 / np.linalg.norm(vec2)
        v = np.cross(vec1, vec2)
        c = np.dot(vec1, vec2)
        
        if np.allclose(c, 1):
            return np.eye(3) 
        if np.allclose(c, -1):
            return -np.eye(3) 

        s = np.linalg.norm(v)
        kmat = np.array([[0, -v[2], v[1]], 
                         [v[2], 0, -v[0]], 
                         [-v[1], v[0], 0]])
        return np.eye(3) + kmat + kmat @ kmat * ((1 - c) / s**2)

    # Calculate normal of the original points
    centroid = np.mean(points, axis=0)
    centered_points = points - centroid
    _, _, vh = np.linalg.svd(centered_points)
    normal = vh[-1]

    # Align normal with Z-axis
    z_axis = np.array([0, 0, 1])
    rotation_matrix = rotation_matrix_from_vectors(normal, z_axis)

    # Transform the projected points
    transformed_points = projected_points @ rotation_matrix.T

    # Normalize the transformed points to fit within the image dimensions
    min_coords = np.min(transformed_points[:, :2], axis=0)
    max_coords = np.max(transformed_points[:, :2], axis=0)
    scale = np.array(img_shape) / (max_coords - min_coords)
    aspect_ratio_scale = np.min(scale)
    offset = (np.array(img_shape) - (max_coords - min_coords) * aspect_ratio_scale) / 2

    # Scale and center points
    pixel_coords = ((transformed_points[:, :2] - min_coords) * aspect_ratio_scale + offset).astype(int)
    pixel_coords = np.clip(pixel_coords, 0, np.array(img_shape) - 1)  # Clip to image bounds

    # Normalize values to fit [0, 255]
    norm_values = ((values - np.min(values)) / (np.max(values) - np.min(values)) * 255).astype(np.uint8)

    # Create image and assign pixel values using advanced indexing
    img = np.zeros(img_shape, dtype=np.uint8)
    img[pixel_coords[:, 1], pixel_coords[:, 0]] = norm_values

    return img

In [19]:
def mean_of_nonzero_neighbors(values):
    nonzero_values = values[values != 0]
    if len(nonzero_values) == 0:
        return 0
    return np.mean(nonzero_values)

def blend_zeros(array):
    zero_mask = (array == 0)
    mean_array = generic_filter(array, mean_of_nonzero_neighbors, size=3, mode='constant', cval=0.0)
    result = np.where(zero_mask, mean_array, array)
    return result


In [20]:
def capture_values(stack, affine_mat, volume_dir, face_points, filter_distance):
    full_verts, _, _, _ = measure.marching_cubes(stack, level=0.5, step_size = 1)
    full_verts = transform_points(full_verts, affine_mat)
    full_verts, _ = filter_points_within_distance(full_verts, face_points, filter_distance)
    vol = np.moveaxis(tiff.imread(volume_dir), [0, 1, 2], [2, 1, 0])
    _, centroid, normal = project_downsampled(face_points)
    planar = project_points(full_verts, centroid, normal)
    detransformed = transform_points(full_verts, np.linalg.inv(affine_mat))
    center = np.array([vol.shape[0] / 2, vol.shape[1] / 2, vol.shape[2] / 2])
    clipped = move_points_towards_center_by_distance(detransformed, normal, affine_mat, center)
    clipped = np.round(clipped).astype(int)
    for i in range(clipped.shape[1]):
        clipped[:, i] = np.clip(clipped[:, i], 0, vol.shape[i] - 1)
    values = vol[clipped[:,0], clipped[:,1], clipped[:,2]]
    return values, planar

In [21]:
def flatten(prob_dir1, prob_dir2, prob_type, filter_threshold, affine_1, affine_2, volume_dir1, volume_dir2,
            export = False, step_size = 10, filter_distance = 50, face_filter_distance = 50):
    """
    Projects the matching faces of two Neuroglancer aligned volumes

    Parameters:
    prob_dir1, prob_dir2: directory of (Ilastik generated) probabilities for volume 1 & 2
    prob_type: file type of probabilities, e.g. 'tiff'
    filter_threshold: int from 1 to 255, filter threshold to create binary mask of probability stacks
    afine_1, affine_2: 4x4 affine matrix for volume 1 & 2
    volume_dir1, volume_dir2: directory of volume tiff files

    step_size: distance between downsampled marching cube points
    filter_distance: distance between volumes; adjust to increase/reduce coverage
    face_filter_distance: radius of downsampled points to capture volume edge

    Returns:
    flat_1, flat2:
    """
    name_1 = prob_dir1.split('/')[-1]
    name_2 = prob_dir2.split('/')[-1]

    stack_1 = initProb(prob_dir1, prob_type)[:,:,:,0]
    stack_2 = initProb(prob_dir2, prob_type)[:,:,:,0]
    stack_1 = filter_probs(stack_1, threshold = filter_threshold)
    stack_2 = filter_probs(stack_2, threshold = filter_threshold)
    verts_1, faces_1, _, _ = measure.marching_cubes(stack_1, level=0.5, step_size = step_size)
    verts_2, faces_2, _, _ = measure.marching_cubes(stack_2, level=0.5, step_size = step_size)
    transformed_1 = transform_points(verts_1, affine_1)
    transformed_2 = transform_points(verts_2, affine_2)
    export_verts(transformed_1, name_1 + '.csv')
    export_verts(transformed_2, name_2 + '.csv')
    filtered_1, filtered_2 = filter_points_within_distance(transformed_1, transformed_2, filter_distance)    
    faces_1_pf = filter_faces_by_points(faces_1, transformed_1, filtered_1)
    faces_2_pf = filter_faces_by_points(faces_2, transformed_2, filtered_2)
    faces_1_af = filter_faces_by_angle(faces_1_pf, transformed_1)
    faces_2_af = filter_faces_by_angle(faces_2_pf, transformed_2)
    points_1 = remove_unreferenced_points(faces_1_af, transformed_1)
    points_2 = remove_unreferenced_points(faces_2_af, transformed_2)
    if (export):
        export_verts(points_1, name_1 + '_final.csv')
        export_verts(points_2, name_2 + '_final.csv')

    values_1, planar_1 = capture_values(stack_1, affine_1, volume_dir1, points_1, face_filter_distance)
    values_2, planar_2 = capture_values(stack_2, affine_2, volume_dir2, points_2, face_filter_distance)

    flat_1 = plot_projection_as_image(points_1, planar_1, (1000,1000), values_1)
    flat_2 = plot_projection_as_image(points_2, planar_2, (1000,1000), values_2)
    flat_1 = blend_zeros(flat_1)
    flat_2 = blend_zeros(flat_2)
    image_1 = Image.fromarray(flat_1)
    image_2 = Image.fromarray(flat_2)
    image_1.save(name_1 + "_flat.png")
    image_2.save(name_2 + "_flat.png")
    
    return image_1, image_2
        

In [22]:
def flatten_stacks(name_1, name_2, stack_1, stack_2, ft1, ft2, affine_1, affine_2, volume_dir1, volume_dir2,
            export = False, step_size = 10, filter_distance = 50, face_filter_distance = 50):
    """
    Projects the matching faces of two Neuroglancer aligned volumes

    Parameters:
    name_1, name_2: names of surfaces to be flattened
    stack_1, stack_2: transformed arrays of probabilities for volume 1 & 2
    ft1, ft2: int from 1 to 255, filter threshold to create binary mask of probability stacks
    afine_1, affine_2: 4x4 affine matrix for volume 1 & 2
    volume_dir1, volume_dir2: directory of volume tiff files

    step_size: distance between downsampled marching cube points
    filter_distance: distance between volumes; adjust to increase/reduce coverage
    face_filter_distance: radius of downsampled points to capture volume edge

    Returns:
    flat_1, flat2:
    """
    stack_1 = filter_probs(stack_1, threshold = ft1)
    stack_2 = filter_probs(stack_2, threshold = ft2)
    verts_1, faces_1, _, _ = measure.marching_cubes(stack_1, level=0.5, step_size = step_size)
    verts_2, faces_2, _, _ = measure.marching_cubes(stack_2, level=0.5, step_size = step_size)
    transformed_1 = transform_points(verts_1, affine_1)
    transformed_2 = transform_points(verts_2, affine_2)
    filtered_1, filtered_2 = filter_points_within_distance(transformed_1, transformed_2, filter_distance)    
    faces_1_pf = filter_faces_by_points(faces_1, transformed_1, filtered_1)
    faces_2_pf = filter_faces_by_points(faces_2, transformed_2, filtered_2)
    faces_1_af = filter_faces_by_angle(faces_1_pf, transformed_1)
    faces_2_af = filter_faces_by_angle(faces_2_pf, transformed_2)
    points_1 = remove_unreferenced_points(faces_1_af, transformed_1)
    points_2 = remove_unreferenced_points(faces_2_af, transformed_2)
    if (export):
        export_verts(points_1, name_1 + '_final.csv')
        export_verts(points_2, name_2 + '_final.csv')

    values_1, planar_1 = capture_values(stack_1, affine_1, volume_dir1, points_1, face_filter_distance)
    values_2, planar_2 = capture_values(stack_2, affine_2, volume_dir2, points_2, face_filter_distance)

    flat_1 = plot_projection_as_image(points_1, planar_1, (2000,2000), values_1)
    flat_2 = plot_projection_as_image(points_2, planar_2, (2000,2000), values_2)
    flat_1 = blend_zeros(flat_1)
    flat_2 = blend_zeros(flat_2)
    image_1 = Image.fromarray(flat_1)
    image_2 = Image.fromarray(flat_2)
    image_1.save(name_1 + "_flat.png")
    image_2.save(name_2 + "_flat.png")
    
    return image_1, image_2
        

In [23]:
# prob_dir_iii = '/Users/jacobliao/Desktop/Kuan Lab/Alignment for Allison 12-19/sect_iii probs'
# prob_dir_iv = '/Users/jacobliao/Desktop/Kuan Lab/Alignment for Allison 12-19/sect_iv probs'
# prob_type = 'tiff'
# filter_threshold = 200
# affine_iv = np.array([[0.946096031445952*4, -0.114345273775416*4, 0.13142091784585*4, 605.708576682309],
#                       [0.116517115059662*4, 0.954884256357037*4, -0.00798867070560139*4, -242.223072148663],
#                       [-0.12949927096472*4, 0.0237742576475981*4, 0.952947387578617*4, -1110.44198682477],
#                       [0, 0, 0, 1]])
# affine_iii = np.array([[4, 0, 0, 0], [0, 4, 0, 0], [0, 0, 4, 0], [0, 0, 0, 1]])
# volume_dir_iii = '/Users/jacobliao/Downloads/2024-12-24_06-31__exm_flat_sect_iii__volume.ome.tif'
# volume_dir_iv = '/Users/jacobliao/Downloads/2024-12-25_21-40__exm_flat_sect_iv__volume.ome.tif'


# flat_iii, flat_iv = flatten(prob_dir_iii, prob_dir_iv, prob_type, filter_threshold, affine_iii, affine_iv, volume_dir_iii, volume_dir_iv,
#             step_size = 10, filter_distance = 250, face_filter_distance = 50, export = True)

In [24]:
# prob_dir_ii = '/Users/jacobliao/Desktop/Kuan Lab/Alignment for Allison 12-19/sect_ii probs'
# prob_dir_iii = '/Users/jacobliao/Desktop/Kuan Lab/Alignment for Allison 12-19/sect_iii probs'
# prob_type = 'tiff'
# filter_threshold = 250

# affine_ii = np.array([[0.982959337590067*4,-0.0632911626542279*4,0.00318894565222629*4,233.275356631831],
#  [0.0632095581678813*4,0.98274679431194*4,0.0209353773747826*4,59.3745214163264],
#  [-0.00452685329098011*4,-0.020687363280016*4,0.984772329322774*4,1152.22395294225],
#  [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  1.0000000e+00]])



# flat_ii, flat_iii = flatten(prob_dir_ii, prob_dir_iii, prob_type, filter_threshold, affine_ii, affine_iii, volume_dir_ii, volume_dir_iii,
#             step_size = 5, filter_distance = 150, face_filter_distance = 50, export = True)


In [25]:
# Rotation matrix is scaled by the downsampling factor. bin2: scale by 2
affine_ii_bin2 = np.array([[0.982959337590067*2,-0.0632911626542279*2,0.00318894565222629*2,233.275356631831],
 [0.0632095581678813*2,0.98274679431194*2,0.0209353773747826*2,59.3745214163264],
 [-0.00452685329098011*2,-0.020687363280016*2,0.984772329322774*2,1152.22395294225],
 [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00,  1.0000000e+00]])

affine_iv_bin2 = np.array([[0.946096031445952*2, -0.114345273775416*2, 0.13142091784585*2, 605.708576682309],
                      [0.116517115059662*2, 0.954884256357037*2, -0.00798867070560139*2, -242.223072148663],
                      [-0.12949927096472*2, 0.0237742576475981*2, 0.952947387578617*2, -1110.44198682477],
                      [0, 0, 0, 1]])

affine_iii_bin2 = np.array([[2, 0, 0, 0], [0, 2, 0, 0], [0, 0, 2, 0], [0, 0, 0, 1]])

prob_type = 'tiff'
filter_threshold = 250

volume_dir_ii_bin2 = '/Users/jacobliao/Desktop/Kuan Lab/Alignment for Allison 12-19/exm_flat_sect_ii_bin2_extended.tif'
volume_dir_iii_bin2 = '/Users/jacobliao/Desktop/Kuan Lab/Alignment for Allison 12-19/exm_flat_sect_iii_tot.tif'
volume_dir_iv_bin2 = '/Users/jacobliao/Desktop/Kuan Lab/Alignment for Allison 12-19/exm_sect_iv_bin2.tif'


In [26]:
prob_iii_bin2 = initStack('/Users/jacobliao/Desktop/Kuan Lab/Alignment for Allison 12-19/probs_sect_iii_bin2d.tif')
prob_ii_bin2 = initStack('/Users/jacobliao/Desktop/Kuan Lab/Alignment for Allison 12-19/probs_sect_ii_bin2d.tif')
prob_iv_bin2 = initStack('/Users/jacobliao/Desktop/Kuan Lab/Alignment for Allison 12-19/prob_sect_iv_bin2.tif')

In [27]:
# Cut off volume at edge of stack for full surface coverage
prob_ii_bin2[:,:,0] = 0
prob_iii_bin2[:,:,6] = 0

In [38]:
flatten_stacks("ii_iii", "iii_ii", prob_ii_bin2, prob_iii_bin2, 200, 250, affine_ii_bin2, affine_iii_bin2, volume_dir_ii_bin2, volume_dir_iii_bin2,
            export = True, step_size = 20, filter_distance = 150, face_filter_distance = 40)

(<PIL.Image.Image image mode=L size=2000x2000>,
 <PIL.Image.Image image mode=L size=2000x2000>)

In [36]:
flatten_stacks("iii_iv", "iv_iii", prob_iii_bin2, prob_iv_bin2, 250, 200, affine_iii_bin2, affine_iv_bin2, volume_dir_iii_bin2, volume_dir_iv_bin2,
            export = True, step_size = 20, filter_distance = 400, face_filter_distance = 40)

(<PIL.Image.Image image mode=L size=2000x2000>,
 <PIL.Image.Image image mode=L size=2000x2000>)

In [239]:
f_prob_iii_bin2 = filter_probs(prob_iii_bin2, threshold = 250)
f_prob_ii_bin2 = filter_probs(prob_ii_bin2, threshold = 250)
f_prob_iv_bin2 = filter_probs(prob_iv_bin2, threshold = 250)

In [209]:
verts_ii, faces_ii, _, _ = measure.marching_cubes(f_prob_ii_bin2, level=0.5, step_size = 20)
verts_iii, faces_iii, _, _ = measure.marching_cubes(f_prob_iii_bin2, level=0.5, step_size = 20)

In [210]:
export_verts(verts_ii, 'sect_ii_bin2.csv')
export_verts(verts_iii, 'sect_iii_bin2.csv')

In [211]:
transformed_1 = transform_points(verts_ii, affine_ii_bin2)
transformed_2 = transform_points(verts_iii, affine_iii_bin2)
filtered_1, filtered_2 = filter_points_within_distance(transformed_1, transformed_2, 150)
faces_1_pf = filter_faces_by_points(faces_ii, transformed_1, filtered_1)
faces_2_pf = filter_faces_by_points(faces_iii, transformed_2, filtered_2) 
faces_1_af = filter_faces_by_angle(faces_1_pf, transformed_1)
faces_2_af = filter_faces_by_angle(faces_2_pf, transformed_2)
points_1 = remove_unreferenced_points(faces_1_af, transformed_1)
points_2 = remove_unreferenced_points(faces_2_af, transformed_2)

In [212]:
export_verts(points_1, 'sect_ii_bin2_final.csv')
export_verts(points_2, 'sect_iii_bin2_final.csv')

In [233]:
values_1, planar_1 = capture_values(f_prob_ii_bin2, affine_ii_bin2, volume_dir_ii_bin2, points_1, 60)
values_2, planar_2 = capture_values(f_prob_iii_bin2, affine_iii_bin2, volume_dir_iii_bin2, points_2, 60)

In [234]:
flat_1 = plot_projection_as_image(points_1, planar_1, (2000,2000), values_1)
flat_2 = plot_projection_as_image(points_2, planar_2, (2000,2000), values_2)
flat_1 = blend_zeros(flat_1)
flat_2 = blend_zeros(flat_2)
image_1 = Image.fromarray(flat_1)
image_2 = Image.fromarray(flat_2)   

In [231]:
image_1.save("ii_iii_bin2" + "_flat.png")
image_2.save("iii_ii_bin2" + "_flat.png")

In [295]:
planar_1.shape

(14456629, 3)